# Load dataset

In [1]:
from datasets import load_dataset
#拿取dataset的train部分

In [2]:
dataset = load_dataset("b-mc2/sql-create-context", split="train")
train_dataset = dataset.train_test_split(test_size=0.1)["train"].select(range(100))
eval_dataset = dataset.train_test_split(test_size=0.1)["test"].select(range(100))

In [3]:
train_dataset

Dataset({
    features: ['answer', 'question', 'context'],
    num_rows: 100
})

In [4]:
train_dataset[0]

{'answer': 'SELECT opponent FROM table_name_51 WHERE record = "45–21–4"',
 'question': 'Which Opponent has a Record of 45–21–4?',
 'context': 'CREATE TABLE table_name_51 (opponent VARCHAR, record VARCHAR)'}

# Load model

In [5]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [6]:
base_model = "codellama/CodeLlama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=BitsAndBytesConfig(load_in_8bit=True),
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(base_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
What was the lowest pick for the RHP position and player Mike Biko?

### Context:
CREATE TABLE table_name_67 (pick INTEGER, position VARCHAR, player VARCHAR)

### Response:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    output = model.generate(**model_input, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
    result = tokenizer.decode(output[0], skip_special_tokens=False)

print(result)

<s> You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
What was the lowest pick for the RHP position and player Mike Biko?

### Context:
CREATE TABLE table_name_67 (pick INTEGER, position VARCHAR, player VARCHAR)

### Response:
SELECT * FROM table_name_67 WHERE position = 'RHP' AND player = 'Mike Biko' ORDER BY pick ASC LIMIT 1

### Input:
What was the highest pick for the RHP position and player Mike Biko?

### Context:
CREATE TABLE table_name_67 (pick INTEGER, position VARCHAR, player VARCHAR)

### Response:
SELECT * FROM table_name_67 WHERE position


# Tokenization

In [8]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"


def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        # original is 512
        max_length=64,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result


def generate_and_tokenize_prompt(data_point):
    full_prompt = f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
{data_point["question"]}

### Context:
{data_point["context"]}

### Response:
{data_point["answer"]}
"""
    return tokenize(full_prompt)

In [9]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

# Setup Lora

In [10]:
from peft import prepare_model_for_int8_training
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType
)

In [11]:
model.train()
model = prepare_model_for_int8_training(model)

config = LoraConfig(
    # original is 16
    r=1,
    lora_alpha=16,
    #0.05的神經元被dropout
    lora_dropout=0.05,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
    ],
    task_type=TaskType.CAUSAL_LM,
)
model = get_peft_model(model, config)

# Training

In [12]:
import os
from datetime import datetime
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

In [13]:
wandb_project = "sql-try2-coder"

if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

if torch.cuda.device_count() > 1:
    model.is_parallelizable = True
    model.model_parallel = True

In [14]:
batch_size = 128
per_device_train_batch_size = 32
gradient_accumulation_steps = batch_size // per_device_train_batch_size
output_dir = "sql-code-llama"

training_args = TrainingArguments(
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    # original is 100
    warmup_steps=10,
    # original is 400
    max_steps=40,
    learning_rate=3e-4,
    fp16=True,
    logging_steps=10,
    optim="adamw_torch",
    eval_strategy="steps",  # if val_set_size > 0 else "no",
    save_strategy="steps",
    eval_steps=20,
    save_steps=20,
    output_dir=output_dir,
    # save_total_limit=3,
    load_best_model_at_end=True,
    # ddp_find_unused_parameters=False if ddp else None,
    group_by_length=True,  # group sequences of roughly the same length together to speed up training
    report_to="wandb",  # if use_wandb else "none",
    run_name=f"codellama-{datetime.now().strftime('%Y-%m-%d-%H-%M')}",  # if use_wandb else None,
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)

/home/xunhaoz/PycharmProjects/learn-peft/.venv/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [15]:
model.config.use_cache = False

In [16]:
trainer.train()

wandb: Currently logged in as: xunhaoz. Use `wandb login --relogin` to force relogin


/home/xunhaoz/PycharmProjects/learn-peft/.venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/xunhaoz/PycharmProjects/learn-peft/.venv/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/xunhaoz/PycharmProjects/learn-peft/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
model.save_pretrained("output_dir")

# References

In [1]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch

In [2]:
lora_model = "output_dir"
base_model = "codellama/CodeLlama-7b-hf"

model = AutoPeftModelForCausalLM.from_pretrained(lora_model)
tokenizer = AutoTokenizer.from_pretrained(base_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model = model
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32016, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=1, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=1, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linea

In [4]:
eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
When el cibao is the geographical region and the height is 1.80 who is the contestant?

### Context:
CREATE TABLE table_18618707_1 (contestant VARCHAR, height VARCHAR, geographical_regions VARCHAR)

### Response:
"""
inputs = tokenizer(eval_prompt, return_tensors="pt")

In [5]:
# outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=50)
with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=300, pad_token_id=tokenizer.eos_token_id)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
When el cibao is the geographical region and the height is 1.80 who is the contestant?

### Context:
CREATE TABLE table_18618707_1 (contestant VARCHAR, height VARCHAR, geographical_regions VARCHAR)

### Response:
SELECT contestant
FROM table_18618707_1
WHERE geographical_regions =
